In [1]:
import os
import numpy as np
import pandas as pd
import csv 
import json
from pandas.io.json import json_normalize


In [2]:
default_path = 'LSC/'
seed = 42
np.random.seed(seed=seed)

In [3]:
def load_data_detection(path):
    all_files = os.listdir(path)
    print(all_files)
    data = pd.read_json(path+all_files[0],convert_dates=False,lines=False)
    data =data.drop(['image-directory'])
    return data
dataFrames = load_data_detection(default_path)
dataFrames


['full_dataset_annotation.json']


,2016-08-15 00:00:00,2016-08-16 00:00:00,2016-08-17 00:00:00,2016-08-18 00:00:00,2016-08-19 00:00:00,2016-08-20 00:00:00,2016-08-21 00:00:00,2016-08-22 00:00:00,2016-08-23 00:00:00,2016-08-24 00:00:00,...,2016-09-01 00:00:00,2016-09-02 00:00:00,2016-09-03 00:00:00,2016-09-04 00:00:00,2016-09-05 00:00:00,2016-09-06 00:00:00,2016-09-07 00:00:00,2016-09-08 00:00:00,2016-09-09 00:00:00,2016-09-10 00:00:00
minutes,"{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Hotel Killarney',...",...,"{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': '', 'latitude': '5...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': '""Park Inn Radisso...",{'0': {'location': {'name': 'Clarion Hotel The...,{'0': {'location': {'name': 'Clarion Hotel The...,{'0': {'location': {'name': 'Clarion Hotel The...,"{'0': {'location': {'name': 'Home', 'latitude'...","{'0': {'location': {'name': 'Home', 'latitude'..."


In [7]:
with open('LSC/full_dataset_annotation.json') as f:
    data = json.loads(f.read())
name,lat,lng,risk_situation,step,gsr,heart_rate,skin_temp,calories,timestamp = [],[],[],[],[],[],[],[],[],[]
for date in data:
    for result in data[date]:
        if(result !='image-directory'): 
            for i in data[date][result]:
                if(len(i)==0):
                    timestamp.append('NA')
                else:
                    timestamp.append(i)
                
                if(len(data[date][result][str(i)]['location']['name']) == 0):
                    name.append('NA')
                else:
                    name.append(data[date][result][str(i)]['location']['name'])
                    
                if(len(data[date][result][str(i)]['location']['latitude']) == 0):
                    lat.append('NA')
                else:
                    lat.append(data[date][result][str(i)]['location']['latitude'])
                    
                if(len(data[date][result][str(i)]['location']['longitude']) == 0):
                    lng.append('NA')
                else:
                    lat.append(data[date][result][str(i)]['location']['longitude'])
                    
                if(data[date][result][str(i)]['image']['risk_situation'] is None):
                    risk_situation.append('NA')
                else:
                    risk_situation.append(data[date][result][str(i)]['image']['risk_situation'])
                    
                if(data[date][result][str(i)]['bodymetrics']['steps'] is None):
                    step.append('NA')
                else:
                    step.append(data[date][result][str(i)]['bodymetrics']['steps'])
                    
                if(data[date][result][str(i)]['bodymetrics']['gsr'] is None):
                    gsr.append('NA')
                else:
                    gsr.append(data[date][result][str(i)]['bodymetrics']['gsr'])  
                
                if(data[date][result][str(i)]['bodymetrics']['heart-rate'] is None):
                    heart_rate.append('NA')
                else:
                    heart_rate.append(data[date][result][str(i)]['bodymetrics']['heart-rate'])
                    
                if(data[date][result][str(i)]['bodymetrics']['skin-temp'] is None):
                    skin_temp.append('NA')
                else:
                    skin_temp.append(data[date][result][str(i)]['bodymetrics']['skin-temp'])
                    
                if(data[date][result][str(i)]['bodymetrics']['calories'] is None):
                    calories.append('NA')
                else:
                    calories.append(data[date][result][str(i)]['bodymetrics']['calories'])
                
df = pd.DataFrame([timestamp,name,lat,lng,step,gsr,heart_rate,skin_temp,calories,risk_situation]).T
df.columns = ["timestamp","name", "latitude", "longitude", "step","gsr","heart_rate","skin_temp","calories","risk_situation"]
print(df.timestamp.values)

['0' '1' '2' ... None None None]


In [ ]:
def truncate(df):
    k = 0
    nb_rows = df.shape[0]
    df_to_fill = pd.DataFrame(columns=df.columns.values.tolist())
    while k < nb_rows:    
        row_considered = df.iloc[[k]]
        phase_considered = row_considered['phase'].values.tolist()[0]
        phase_tmp = phase_considered
        #print(phase_considered)
        #print(phase_tmp)
        i = 0
        while phase_tmp == phase_considered and i<30:
            if k + i + 1 < nb_rows:
                # Verifying if there is a next value
                i += 1
                phase_tmp = df.iloc[[i+k]]['phase'].values.tolist()[0]
            else:
                # End of while
                break
        if i > 7:
            # Add the row considered            
            df_to_fill = df_to_fill.append(row_considered)            
        if k + i + 1 < nb_rows:
            k += i
        else:
            # End of the dataframe
            break
    df_to_fill = df_to_fill.reset_index(drop=True)
    return df_to_fill
df_truncated = truncate(dataFrames)
#print(df_truncated.shape)
#print([df_truncated['phase'].loc[lambda phase: phase==header].count() for header in headers])
#print(df_truncated)

In [ ]:
def creat_missing_values(df, nb_rows_to_del=1):
    nb_rows = df.shape[0]
    nb_columns = df.shape[1]
    index_still = np.arange(nb_rows)
    #print(index_still)
    df_copy = df.copy()
    if nb_rows_to_del < nb_rows:
        while nb_rows-len(index_still)<nb_rows_to_del:
            index_del = np.random.randint(len(index_still)-1)
            timestamp_saved = df.iloc[index_del].__getitem__('timestamp')
            #print(timestamp_saved)
            df_copy.iloc[index_del] = [np.nan for i in range(nb_columns)]
            #print(df_copy.iloc[index_del])
            df_copy.loc[index_del, 'timestamp'] = timestamp_saved
            #print(df_copy.iloc[index_del])
            index_still = np.delete(index_still,index_del)
    return df_copy
df_copy = creat_missing_values(df_truncated, 5)
#print(df_truncated.shape)
print(df_copy.head(40))

In [ ]:
df_copy.isna().count('lhx')

In [ ]:
timestamp_saved = df_truncated.iloc[0].__getitem__('timestamp')
print(timestamp_saved)
df_nan = pd.DataFrame(columns=df_truncated.columns.values.tolist())

df_nan = df_nan.append([np.nan])
df_nan.loc[0,'timestamp'] = timestamp_saved
print(df_nan)

In [ ]:
df_truncated.iloc[0].__getitem__('phase')


In [ ]:
keys = ['lhx', 'lhy', 'lhz', 'rhx', 'rhy', 'rhz', 'hx', 'hy', 'hz', 'sx', 'sy', 'sz', 'lwx', 'lwy', 'lwz', 'rwx', 'rwy', 'rwz', 'timestamp', 'phase']
input_keys = ['lhx', 'lhy', 'lhz', 'rhx', 'rhy', 'rhz', 'hx', 'hy', 'hz', 'sx', 'sy', 'sz', 'lwx', 'lwy', 'lwz', 'rwx', 'rwy', 'rwz']
timestamp_key = 'timestamp'

TODO:
Concatenate all datas,
truncate in this dataset,
make random missing data

Run.py : training / test

In [ ]:
timestamp = df_Big['timestamp']
processed_data = {
    'input': [],
    'masking': []
}
n = len(timestamp)

In [ ]:
def creat_mask(timestamp, input_data):
    

In [ ]:
timestamp